# 전체 크롤링 코드

In [1]:
import os
import csv
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')
chromedriver_path = "C:/chromedriver"
driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기

reviews = []  # 중복 리뷰를 저장하지 않기 위한 리스트 생성

def main():
    global driver, load_wb, review_num

    driver.implicitly_wait(4)
    driver.get('https://map.kakao.com/')

    # 검색할 목록
    place_infos = [
        "제주도 해수욕장",
        "제주도 산림욕장",
        "제주도 녹차밭",
        "제주도 박물관",
        "제주도 오름",
        "제주도 놀이공원",
        "제주도 미술관",
        "제주도 수목원",
        "제주도 테마파크",
        "제주도 카페",
        "제주도 물놀이",
        "제주도 수영장",
        "제주도 동물원",
        "제주도 승마장",
        "제주도 공원",
    ]

    for i, place in enumerate(place_infos):
        if i % 4 == 0 and i != 0:
            sleep(5)
        print("#####", i)
        search(place)
        
    save_to_csv("제주도", reviews)
        
    driver.quit()
    print("finish")


def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        # 2~ 5페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(place, place_lists[:5])

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()

def save_to_csv(place, reviews):
    file_name = f"place_reviews.csv"
    with open(file_name, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['place_name', 'review']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for place_name, review in reviews: 
            writer.writerow({'place_name': place_name, 'review': review})
            print(f"{file_name} saved.")



def crawling(place, place_lists):

    while_flag = False
    for i, place in enumerate(place_lists):
        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print(i+1,'.', place_name)

        # 첫 페이지
        extract_review(place_name)

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 리뷰 페이지 찾기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('후기 더보기').send_keys(Keys.ENTER) # 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        crawl_next_reviews(place_name)

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환

def extract_review(place_name):
    global driver, reviews
    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    review_lists = soup.select('.list_evaluation > li')

    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span')
            if comment:
                comment_text = comment[0].text.strip()
                if comment_text and (place_name, comment_text) not in reviews:  # 수정된 부분
                    reviews.append((place_name, comment_text))  # 수정된 부분
                    print(comment_text)
    else:
        print('no review in extract')
        ret = False

    return ret

def crawl_next_reviews(place_name):
    global driver
    max_clicks = 10  # 최대 클릭 횟수 설정
    click_count = 0 # 현재 클릭 횟수 초기화
    while click_count < max_clicks:
        try:
            driver.find_element_by_link_text('후기 더보기').send_keys(Keys.ENTER)  # 후기 더보기 버튼 클릭
            sleep(1)
            if not extract_review(place_name):  # 리뷰 추출
                break
            click_count += 1  # 클릭 횟수 증가

        except (NoSuchElementException, ElementNotInteractableException):
            print("no more reviews to load")
            break

            
            
if __name__ == "__main__":
    main()
    

##### 0
1 . 이호테우해수욕장
꼭 건강 해져서 다시한번 가고싶다 병원 밖으로 나가서  제주도로 가자
이호는 선셋이 너무아름다운 곳이에요. 주차공간이 조금 협소해서 아쉬워요.
바다 색 미쳤누바람이 엄청 많이 불었는데 사진은 이쁘게 잘 나왔던
말 필요 없음
이호테우 해변 비오는 날 방문했는데 운치있고 넘 좋네요!!!!
좋았어요 ㅎㅎ 말등대도보공
귀여운 말등대가 있는 해수욕장🪅아이들 놀기 좋은 해수풀장이 있어요~
제주도 마지막 여행지
그냥 시원한 맛에 갈만함 사람 꽤 있는 편이고 말 등대까지 산책하기 괜찮음 -겨울방문- 여름에는 해수욕 하기 괜찮을듯 잔잔하니 아이들이 놀기 괜찮음
https://youtu.be/R4hXLXd9s9Aok차박 이호태우해수욕장 유튜브 검색하시면 동영상 올렸습니다 낮과밤 차박지
서핑스팟으로서0점. 라인업에 사람 30명중문처럼 꼽주는 쓰레기 다수물론 좋은사람이 많지만까맣고 얼굴 찌뿌리고 있는 사람 많은 해수욕장에는서핑하러 가는거 아님
물이 맑아서 속이 다 보이는 곳!말 동상으로 유명해서 와보고 싶었던 곳!!플라스틱에 대한 경각심을 불러일으키는 곳!
인생샷 찍으러왔니? 응 여기아니야..처음가봤는데 말등대가 왜이리 멀어.. 이호항 으로 가야댐 가즈아!
제주도 바다 중 물색은 가장 안 좋은듯...
알바생들이 손님 뒷담까면서 욕하네요^^ 담부터는 안갈듯요~ 흔한 일진알바생인지  불쾌하네용^^
좋았음
제주바다중에 가장더러운 해수욕장?  절대 가지마시길 가실거면  협재  김녕 함덕 표선이  제일  좋고 깨끗함
좀 걷긴 해야하지만 버스 정류장이 근처에 있어서 접근성이 좋았어요 하지만 물이 좀 더럽고(해조류 많음!) 화장실에 냄새가 났어요 ㅜㅜ 그래두 가볍게 물놀이 하기엔 좋아용!!
예쁜 뷰가 보이는 항구...
물더러움 냄새남애기들은 이호테우 해수풀장에서 노세용
관리가 제대로 안되고 있는 거 같아요;; 서핑하러 갔다가 물이 더러워서 한시간 만에 나왔어요물에 비닐봉지도 너무 많이 떠다녀서 충격먹고 나왔습니다....
애랑 가서 놀기좋아요 물이 깨끗하진 

어른은 입장료 2천원아이들과 함께 멋진 뷰를 볼 수 있어 너무 좋았어요9:00-3:30까지 관람 가능하고 날씨가 안좋은 날은 관람이 어렵다고 하니 참고하시면 좋을 거 같아요부모님도 아이들도 모두 만족했어요비싼 관람료내고 대부분 관광지 많이 구경 다녔는데 여기는 입장료도 저렴하고 포토 스팟도 많아 좋았어요
정말 멋지다! 친구들과 여행에 내가 추천했는데 다행히 입장할 수 있었고. 생각보다 코스가 길어서 더 좋았다. 여긴 또 가고싶다.
산방연대 간 김에 용머리해안까지 봐줘야 만족스러움! 너 무 예 쁨!!!!!
멋진 사진을 찍을 수 있는 곳!
파고가 높아서 종일 통제에 걸림. 1차실패ㅠㅠ (진짜 파도가 잘못하면 죽을거같이 보이긴 하더라;;;) 담엔 꼭 관람 성공할거에요.
제주여행에서 제일 기억 남는곳이네요...인생샷 많이 찍었어요. 꼭! 끝까지 가세요 출구있어요. 초입보다 뒤쪽이 더 멋있습니다. 할머니들이 파시는 해산물은 맛은 봤지만 솔직히 바가지인거 같아요.
배경은 매우매우 좋음.. 그러나 왜 아무도 투명바퀴벌레같이 생긴 그 벌레언급들을 안해준건지.평생 도시에서 살다온 나로써 그 벌레 우글우글 거리는걸보고 도저히 더 보고 올수가 없었다. 리턴해서 다시 나감.. 내가 간 날만 그런건지.. 과장없이 한번에 몇백마리 집단이 움직이는걸 보고옴
최고입니다 제주도에 가서 여길 안 보면 제주도의 절반만 본 거라고 말해도 될 정도
기암괴석이 너무 멋집니다~ 이런 절경을 우리나라 다른 곳에서 찾아보기 힘들듯~
정말 멋지네요 ㅎㅎ 3살아기도 엄마 아빠 도움 받아서 구경 잘 했어요 ㅎㅎ
가보시는 분들은 끝까지 가보세요~ 저는 가족들이랑 있어서 끝까지 못갔는데 마지막이 찐이라고 하시더라고요.. 아쉬워서 다음에 끝까지 가보려고 합니다~
내가 너무나도 좋아하는 곳자연이 만든 경관에 계속 감탄하게 된다. 이곳에서 바라보는 산방산도 최고!!중간에 돌아가지 않고 꼭 출구까지 주욱 걷는걸 추천
무조건 가야 하는 곳
최고 멋있다bb 제주올때마다 들리고 싶을 만큼…만조시간도 매일 다르고 파도가 쎄면 

낫베드입니다
깨끗한 용천수가 계속 공급되어 물도 맑고 덜 짜서 좋고 차박하기 조용합니다
수영장x, 화장실 위생최악 안들어가는게 정신건강에 좋음, 관리x,  그냥 바다 풍경보기엔 좋아요.
리뷰사진보고 친구들하고 다같이갔는데 왠걸;;왠 고인물있는 곳이네요..안에 쓰레기도 있었고 이끼때문에 바닥은 디게미끄러워요.. 이상적인 해수욕장보단 그냥 수영장에가까운 느낌..
똥물.... 아쉽 ㅠㅠ
시원해요~~~
들렁궤까지 잠깐 걸음. 들렁궤쪽이 도민 낚시 포인트인거같다. 돌이 판판하게 잘 나있는데 쭉이어져있다. 모래사장아니고 돌바닥. 바다가 깔끔한편이다. 논짓물에는 바다에 인피니트 수영장처럼 만들어 놓은 곳이 있는데 여름에 자릿세를 받고 빌려주는 것 같다. 특별하진않고 그냥 숨겨진 도민 포인트같아보임
날씨 풀리면 꼭 다시와서 수영하고싶어요ㅠ
갈대랑 바다 너무 이쁘네용
좋아요
민물과 바닷물이 함께하는곳 강추입니다.
3단 수영장, 아쿠아슈즈 필수. 물이 아주 시원합니다.
지나가다 우연히 발견. 여름에는 수영도 할 수 있는거 같아요. 한적하니 좋아요 부대시설도 나름 괜찮음
no more reviews to load
5 . 검멀레해변
모래가 빛을 흡수해서 따땃하고 좋당
사진봐도 별 감흥이 없어서 기대가 없었는데사진에 못담는거였네여 너무 멋있었습니다
23개월 여아 동반순환 버스를 타고 급하게 구경 하였습니다 경관이 너무 이쁘고 좋았지만 비오고 바람이 많이 불어 힘들었어요 ㅎ
경관은 너무 아름다운 곳입니다. 쓰레기가 좀 많이 방치되어 있다는게 한가지 아쉬운 점입니다.
멋진 해변입니다. 보트가 재밌긴 하겠지만, 계속 한 가운데 가만히 떠있어서 경치를 해치네요. 운행 안 할 때는 선착장에 있었으면!
서빈백사 이후로 좋아하는 두번째 해변검은 모래가 너무 신기했던듯 함옛날에는 별거 없었지만 지금은 카페도 많이 생기고 활기 넘침
Stop for a peanut ice cream at Jimmys and enjoy the cliffside views
https://youtu.be/wJBM7jd_4

가볍게 산림욕하기 좋은 곳 흔한 한국 숲하고는 조금 다른 경치
제주도 왔다면 사려니숲길과 더불어 들러야한다 생각함.제주도가 왜 환상의 섬인지 다시금 느끼게됨너무 좋은곳 어른들이랑도 산책하기 좋은 곳
2022.10.08 마님과
너무너무 좋았어요 제주도 여행마다 가고싶은 곳
오래된 나무들과 새소리 향긋한 피톤치드향이 맞이하는 곳. 사람이 좀 많으서 주자장은 약간 부족한듯요
가볍게 산책하기 좋은 곳 입니다 :)
성인 둘 6천원 17시 이전까지만 입장 가능 17개월 아기와 대충 1시간 정도면 왕복 시설은 더 좋아 진것 같고 옛날 그대로 관리 잘되어 있음
제주도에서 가장 좋았던 곳입니다. 햇볕이 내리쬐는 비자림을 걷고있으면 판타지속 엘프의 숲을 걷는 것 같고 어딘가에서 오래된 유적이 튀어나올 것 같아요. 한국에서 흔하지 않은 수형을 가진 큰 나무들이 이렇게 많이 있는 곳이 드물것 같습니다. 말은 거창하게 해뒀지만, 가보면 그냥 향기가 좋은 숲이긴 해요. 길을 잘 가꿔놔서 찬찬히 산책하기 좋습니다. 바다보다 숲이 좋다는 분들에게 강력추천합니다.
비자림 좋네... 정각마다 안내 가이드분이 돌아다닙니다 근데 ㄹㅇ 발 개빠름 끝날때 즈음 되면 남아있는 사람이 없음 하지만 안내 들으면서 가는게 낭만이죠. 걷기좋고 바닥에 뽀작뽀작 소리 짱 ㅋㅋ
비자림 오실 때 향수 뿌리고 오지 말아주세요ㅠㅠ비자나무 냄새 맡으러 갔는데 옆사람 향수냄새 진짜 최악…
좋은 공기 맡으러 가는 곳
2 . 서귀포자연휴양림
숲길이 참이쁩니다 산책하기에너무좋와요
제주도에서 운전해서도 갈 수 있는 휴양림 /야영장규모도 생각보다 크며 깔끔하다
숲길산책로(야자매트 및 흙길) - 법정악전망대(데크길) - 생태관찰로(데크길) 코스로 돌아나온 후 무장애나눔숲길(데크길)까지 걸었고, 총 2시간반정도 걸림.길 표시도 잘 되어있고, 중간중간 캠핑사이트도 있어서 혼자 걸어도 전혀 위험하지 않음.지대가 높아서인지 이제 막 피기 시작한 동백꽃 꽃봉오리들도 있음(23.04.03기준)난이도도 평이한 수준이라 부담없이 걷기 좋음.위에 

1박 숙박하였는데 밤에 쏟아지는 별들은 평생 잊지 못할 정도로 아름다웠고 조용하게 잡념 털어내기 너무 좋았습니다. 다만 주차장과 숙소까지 차량 진입이 불가하고 10여분 도보로 걸어야해서 일몰 이후 왔다갔다하는데 상당히 곤란합니다. (가로등은 있으나 매우 어둡고 동물들이 출몰합니다.)
지린다
조용하게 산책 할수 있는곳.
좋음  아침에 오름 오를수도 있고..
산책하기 좋은 곳.
매표소 이모님이 진짜 사람 무안하게 하시더라구요. 매우 불친절함을 지나 안으로 입장.  평평하고 아이들 데리고 산책하기 좋아요. 까마귀 진짜많쿠요.
오름만 올라가려고했는데도 입장료를 받아요ㅠ 주차비도... 오름 가실분은 다른곳으로가세요
4인용 숙소 이용. 시설은 잘되어 있나 겨울은 추움. 에어컨 겸용 히터까지 틀어야 함. 호텔에 비해 가성비 낮음.  친환경휴양림을 표방하고 있어 오름 탐방로와 전체적인 시설은 잘 되어 있으나, 직원들의 프라이드가 상당히 높아서인지 친절을 기대하기 어려움. 숙소는 주차장에 차를 세워두고 100~200m가량 가야해서 짐이 많은 관광객은 매우 불편함.붉은오름은 스코리아가 퇴적되어 생긴 오름으로 산화에 의해 붉은색을 띄어 붙여진 이름이라고 함. 실제 스코리아는 관리실 옆 마당에 전시되어 있는 정도이고, 오름을 오르면서는 보기 어려움. 전형적인 곶자왈이 나타나지 않아 아쉬웠음. 오름 하부의 탐방로 숲길이 아름다움. 정상까지만 다녀와도 좋음. 정상까지 왕복1시간 이면 충분함.
비누나 수건 등도 없어서 아쉽고 쓰레기도 가지고 나와야된다는건 몰랐네요. 가격대비는 괜찮았어요..
휴양림치고 침구류가 뽀송뽀송하게 준비되있어서 매우 안심하고 잘수있었다. 다만 수건은 고사하고 치약, 샴푸, 심지어 비누조차 준비되어 있지 않은 점은 무척 아쉽다.쓰레기는 분리수거해서 배출하도록 되있으면서 종량제 봉투나 비닐봉지 조차 제공안하는 것은 또 무언가? 관리소 입구에서 부터 숙소까지 차량출입을 통제하는 것은 좋다. 휴양림을 청정하게 유지한다는 의미에서는. 헌데 비오거나 늦은 밤 짐이라도 잔뜩 

...

공연 참 재미있습니다.젙말 볼만하네요.카트나 다른건 비싸서 못탔어요.
말탄 사진 값을 좀내리면 좋을텐데요...너무 비쌈..마상 공연 강추입니다.멋진 공연을 위해서더운데 배우들 넘 고생인듯요.꼭 봐야해요!!!
기마공연 최고에요
마상쇼 봤어요. 잼나요 득탬
재미있어요
제주에 올때마다 가는곳! 바람 겁나 불고 추운 날이라서 그런지 연기하는 분들이 애처로워 보임 꼭 한번은 보시기를 추천!
카트하고 승마 단거리했습니다! 카트는 9900원  돈안아깝게 정말재밋게 탔고 승마는 7900원 제주도오셨으니까 말한번타고 가세요..재미는 그냥그렇습니다ㅜ 제몸무게버티느라..말이..저랑같은생각하시는분 많을꺼에요ㅎㅎ
승마도재밌고 카트는더잼나고다음에 꼭 공연볼께요~~
쇼쇼쇼 돈벌이
공연만 보고 다른 체험은 하지 않은 평가입니다.공연 너무 재밌어요. 연기하시는 분들도 호응 끌기 위해 부단히 노력하셨고, 무엇보다 말들이 연기하는 모습이 너무 신기해서 넋을 놓고 봤네요. 다만... 말들에게 당근주기할때, 금액대비 당근이 너무 작다는... 당근을 8천원어치나 사서 고생한 말들 먹이고 왔는데도 너무 부족해보였어요. 당근 크기를 좀 더 키워주던지.. 가격을 좀 다운시켰으면 하네요. 그 점이 아쉬워서 별 하나 뺐어요.
카트 진짜 재밌어요 대신 사람 없을 때 타야 최고속도로 코너링 즐길수 있습니다카트 2인용 35000원에 타고 동물원 30프로인가 할인받아서 구경했는데 규모는 작았지만 체험형이라서 예상외로 너무 재밌었어요사람이 별로 없어서 사육사들이 먹이도 챙겨주시고 살다살다 앵무새와 교감해본적은 처음이었습니다 말을 정말 잘하구 사람을 좋아하더라구요
기대 못했는데 스케일도 크고 연기도 잘하시고 재미있었습니다.
기대하고 가지 않았는데 공연이 꽤나 스케일이 크다.(20여마리 정도?) 내용 자체가 흥미로운건 아니었지만 말과 함께 공연을 저렇게도 할 수 있구나 재밌게 하면서 봤음.
관람시간 맞춰서 가면 좋아요!말타기 체험은 그냥A코스 추천드릴게요
공연은 강추승마는 넘 오래 기다리게 해서 비추
공연은 좋았으나 추

no more reviews to load
2 . 홀스스토리승마장
기분 좋게 만드는  곳
나혼자산다나온 곳
no review in crawling
no more reviews to load
3 . OK승마장
남자직원 두분은 말하시는게 참 기분나쁘게 말하시던데 일하기 싫으면 걍 집에가시던가요 손님들 짜증나게 하지말고
친절히 안내해 주시고 쾌적한 설비에 넉넉한 승마시간으로 초등5학년 따님과 아빠 멋진 풍경 즐기면서 소중한 추억과 사진 잘 챙겨갑니다~^^다음 제주도 여행에 꼭 다시 들르고 싶습니다~!!
어투는 친절한 느낌은 아니었어요. 아마도 제주도 억양? 아이에게 주는 지적도 좀 아쉬웠지만...스타일이겠지요.  내용은 좋았어요. 가기 전 세군데 전화해도 다 마찬가지로 형식적인 응대.딸과 즐겁게 탔고, 딸도 좋아 했어요. 30분짜리 충분히 길어요. 본심이 잘 전달되도록 아주머님 억양도 조금만 고쳐지고, 도시분들도 입맛에 딱 맞는 기대를 쫌만접으면...까페에서 일하는 영혼없는 알바보단 훨 나았음.
Advanced Climbing Horse Riding
no more reviews to load
4 . 조은승마장
사장님은 친절하신데 목장 2바퀴정도 말타고 앉아서 도는게 끝이에요. 긴 코스가 가능한 날이 아니면 일부러 찾아갈 정도는 아닌것같아요
아이들 승마체험하기 적당합니다.
직원 분들 모두 친절하세요^^ 풍경도 좋았고 거리나 속도 모두 적당했습니다 말들도 너무 순하고 귀여워요☺ 산책코스 타시는 것 추천해요~
대체적으로 직원분들이 다 친절하시고 가격도 다른 곳에 비해 저렴해서 좋았고 제일 긴 코스는 사진 무료로 찍어 인화해주시는데 서비스가 좋은 것 같아요
친절하시구 사진도 무료에 좋아요
범퍼카 탈만합니다
처음 타는 사람들에게 체험하기에 가성비면에서 좋아요
네이버예약해서 저렴히 다녀왔어요~ 친절하시고 말들도 순하고 사진 1장 무료이고 좋았어요~ 다만 산책코스 달리기 좀만 더 해주시면 좋을듯요^^
땡볕에 말들이 다 묶여있고 아스팔트위에 한마리 갇있는데 왜 갇혀있는지 모르겠지

2 . 환상숲곶자왈공원
숲해설을 들으면서 곶자왈에 대해 알게되었는데무엇보다 식물들의 신비한 모습들을 직접 보고 배울 수 있어 좋았어요
해설은 나쁘지 않으나 도립공원에 비해 너무 협소.
숲 해설 나쁘지 않아요. 다만, 너무 볼 게 없어요 … 걷는 코스도 그닥 ; 그에 비해 입장료 5,000₩ 너무 비싸네요. 서사에 약한 한국인들 자극은 잘 한 듯 하나 … 리뷰에서 극찬 하는 정도는 절대 아닙니다.
넘 좋았음
숲해설 꼭 들으세요. 최고입니다
숲해설 꼭 들으세요!! 같이 해설 들었던 분들 중에 해설이 너무 재밌어서 일주일전에 오시고 또 들으셨던 분들이 계셨어요, 사회생활에 치여 인류애가 사라졌었는데 완전 채우고 왔어요! 🌱🍀🏜️🌼족욕하면서 차도 마시고 진짜진짜 좋았어요그리고 소금스크럽!! 완전 강추예요나중에 아이 생기면 꼭 같이 갈거예요
환상숲 그대로 입니다자연힐링이 되는곳 이고요 강추합니다.꼭 숲해설사의 설명을 들으면 더 좋읍니다
여긴 찐입니다. 꼭 가세요. 해설 무조건 들으셔야 합니다. 덕분에 숲에 대해 많이 배우고 우리들 인생도 비슷하구나 했습니다. 좋은시간 보냈습니다. 감사합니다.
힐링하고 갑니다 숲해설 꼭들으세요!
볼것이하나도 없네요.입장료는 오천원이나 받으면서 너무실망했어요.
https://youtu.be/p_z0pjxky6kok차박 환상숲 유튜브 검색하시면 동영상 올렸습니다 주차장넓고 화장실깨끗 해설사님의 마음다듯한 전달 사랑의 나무 덩굴나무 칡나무의 번되 주변에 분위도 좋네요
숲 해설 듣지 않을거면 가지마세요.꼭 들으세요!부모님, 아이들과 가볍게 산책하며 보고 듣기 좋은 곳.숲을 보고 왔는데 마음이 따뜻해지는 곳.
걷기좋은 나들이 이야기가 아름다운 곳다시 찾고 싶은곳마음과 몸 머리가 건강 해지는 곳응원합니다
2022.10.07 마님과
그냥 산책만 하면 이곳의 특별함을 몰랐을텐데 해설가님의 재밌는 해설과 함께하니 숲이 더 의미있게 느껴졌어요 코스가 짧다는 평이 있는데 최근에 1.2km산책로도 새로 열렸다고 하네요! 좋은 추억 남기고 갑니다
그냥 다른 곶

새섬을 한바퀴 돌게 되어 있어서 더좋았어요
환상적이었어요.
시간 남아서 우연히 갔는데너무 좋았음
사실 시간때우러 간건데 넘나 좋아요. 기대이상!
서귀포엔 보통 천지연폭포를 보러 많이들 오지만새섬공원이야 말로 서귀포 앞바다와 멋진 한라산뷰를 제대로 느낄 수 있는 곳새연교를 건너 한바퀴 도는데 그리 오래 걸리지 않으며 남녀노소 누구나 쉽게 다닐 수 있게 길도 잘 정비돼 있다
돈 안들고 경치 좋고 참 좋다.
낮에 가시면 서귀포항을 느끼며 산책할수있어요1키로정도의 짧은코스인데 원없이 바다를 볼수있답니다핫한 포토스팟이 있는건 아니지만전체적으로 평화로운 분위기라소박하게 다녀오기좋아요
야경보러갔는데 이쁨
나의 프로포즈 장소언제나 내 마음 속의 1번바람이 많이 불어서 추울 수 있어요!
no more reviews to load
2 . 제주밭담테마공원
조성한 이유를 잘 알 수는 없음. 위치가 매우 애매함.
뭐하는덴지 모르겟음
pleasant place to take a walk out onto a small volcanic island connected by a concrete seawall. there are see-through kayaks available for 10,000₩ per 30 mins. upside: few people, so it's quite quiet downside: not much shade on a sunny day, and few food options
no more reviews to load
3 . 제주돌마을공원
기대 안하고 왔는데 엄청 좋아요 사장님도 착하시구요
무난하게 구경할만해요. 지형이 험난해서 운동화신고 오세요
1시간동안 제주도를 다 볼수있어요.  동물원도 있구요..ㅋㅋㅋ
돌 밟으면서 돌 보면서 산책하기에는 괜찮은데..어르신들은 좋아할듯젊은이들 취향은 절대 아님그리고 돌멩이 전래동화가 구시대적임 바람피는 남자를 벌하기위해서 딸만 줄줄이 낳게하고 남자가 반성하니 신이 용서하며 아들을 포상으로 내리는 성차별적 스토리.. 옛날이야기니까 

finish


In [2]:
import os
from time import sleep
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')
chromedriver_path = "C:/chromedriver"
driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)

def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')
    search_area.send_keys(place)
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)
    sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem')

    crawling(place, place_lists[:5])
    search_area.clear()

    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        for i in range(2, 6):
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem')

            crawling(place, place_lists[:])

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()

def crawling(place, place_lists):
    global driver, all_data

    for i, place in enumerate(place_lists):
        place_name = place.select('.head_item > .tit_name > .link_name')[0].text

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])
        sleep(1)

        print(i+1, '.', place_name)        
        
        data_dict = extract_data()

        if data_dict:
            all_data.append(data_dict)

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

def find_element(xpath):
    global driver

    try:
        element = driver.find_element_by_xpath(xpath)
        return element.text
    except NoSuchElementException:
        return ""

def extract_data():
    global driver

    place_name = find_element('//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2')
    place_location = find_element('//*[@id="mArticle"]/div[1]/div[2]/div[1]/div/span[1]')
    place_feature = find_element('//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/span[1]')
    place_star =  find_element('//*[@id="mArticle"]/div[5]/div[1]/div/em')
    operating_time = find_element('//*[@id="mArticle"]/div[1]/div[2]/div[2]/div/div[1]/ul/li/span/span[1]')

    if place_name:
        data_dict = {"관광지 이름": place_name,
            "place_name": place_name,
            "place_location": place_location,
            "place_feature": place_feature,
            "place_star": place_star,
            "operating_time": operating_time}
        return data_dict
    else:
        return None

def main():
    global driver, all_data
    all_data = []
    driver.implicitly_wait(1)
    driver.get('https://map.kakao.com/')
    
    place_infos = [ "제주도 해수욕장",
        "제주도 산림욕장",
        "제주도 녹차밭",
        "제주도 박물관",
        "제주도 오름",
        "제주도 놀이공원",
        "제주도 미술관",
        "제주도 수목원",
        "제주도 테마파크",
        "제주도 카페",
        "제주도 물놀이",
        "제주도 수영장",
        "제주도 동물원",
        "제주도 승마장",
        "제주도 공원",]
#     place_infos_str = "".join(place_infos)

    for i, place in enumerate(place_infos):
        if i % 4 == 0 and i != 0:
            sleep(1)
        search(place)

    driver.quit()
    print("finish")

    # Save all_data to a CSV file
    if all_data:
        data_frame = pd.DataFrame(all_data)
        data_frame = data_frame[['관광지 이름', 'place_name', 'place_location', 'place_feature', 'place_star', 'operating_time']]
        data_frame.to_csv(f"place_data.csv", index=False, encoding='utf-8-sig')
        print("Data saved to top_25_places_data.csv")
    else:
        print("No data to save")

if __name__ == "__main__":
    main()

1 . 이호테우해수욕장
2 . 남원큰엉해변
3 . 함덕해수욕장
4 . 협재해수욕장
5 . 용머리해안
1 . 황우지해안
2 . 삼양해수욕장
3 . 행원해변
4 . 논짓물해변
5 . 검멀레해변
6 . 화순금모래해수욕장
7 . 우도산호해변
8 . 하도해변
9 . 신창풍차해안
10 . 하고수동해변
11 . 엉알해안
12 . 신양섭지해수욕장
13 . 평대리해수욕장
14 . 종달리해변
15 . 하효쇠소깍해변
1 . 몽돌해변
2 . 세기알해변
3 . 신흥해수욕장
4 . 하모해변
5 . 알작지해변
6 . 소금막해변
7 . 광치기중간
8 . 황우치해변
9 . 우뭇개해안
10 . 톨칸이해변
11 . 수마포해안
12 . 설쿰바당
13 . 모진이몽돌해변
14 . 삼양해수욕장종합상황실
15 . 후포해변
1 . 함덕해수욕장 주차장
2 . 협재해수욕장 주차장
3 . 협재해수욕장 캠핑장
4 . 곽지해수욕장 주차장
5 . 세화해수욕장 주차장
6 . 이호테우해수욕장 주차장
7 . 표선해수욕장 야영장
8 . 함덕해수욕장 야영장
9 . 김녕해수욕장 야영장
10 . 금능해수욕장 야영장
11 . 협재해수욕장 야영장
12 . 중문색달해수욕장 공영주차장
13 . 곽지해수욕장 옆 주차장
14 . 이호해수욕장 공영주차장
15 . 화순금모래해수욕장 수영장
1 . 김녕해수욕장 야영장주차장2
2 . 삼양해수욕장 정자
3 . 삼양해수욕장 공영주차장
4 . 하도해수욕장 화장실
5 . 이호테우해수욕장 화장실
6 . 표선해수욕장 샤워장
7 . 함덕해수욕장 아영장 공중화장실2
8 . 화순해수욕장 캠핑장
9 . 화순금모래해수욕장 주차장
10 . 파리바게뜨 함덕해수욕장점 전기차충전소
11 . 서빈백사해수욕장 주차장
12 . 곽지해수욕장 화장실
13 . 함덕해수욕장 야영장 공중화장실1
14 . 하모해수욕장 주차장
15 . 금능해수욕장 주차장 전기차충전소
1 . 비자림
2 . 서귀포자연휴양림
3 . 절물자연휴양림
4 . 붉은오름자연휴양림
5 . 교래자연휴양림
not found
1 . 설록다원도순
2 . 서귀다원
3 . 녹차동굴

13 . 어울공원 실외수영장
14 . 라마다프라자 제주호텔 수영장
15 . 옥돔역 태흥물놀이장
1 . 담앤루 수영장
2 . 휘닉스제주섭지코지 수영장
3 . 마레보리조트 수영장
4 . 다인리조트 풀장
5 . 메종글래드제주 더파티오풀
6 . K&D스포츠 제주영어교육도시점 수영장
7 . 삼성초등학교 수영장
8 . 유키즈풀 어린이수영장
9 . 대정중학교 수영장
10 . 선린지리조트 수영장
11 . 이에스수영교실
12 . 표선중학교 수영장
13 . 야외풀장
14 . 베니키아홈더제주리조트 야외수영장
15 . 해비치호텔앤리조트 실외수영장
1 . 뉴코리아유스호스텔 수영장
2 . 피크포인트
3 . 제주신라호텔 수영장
4 . 용담어린이집수영장
5 . 라마다프라자 제주호텔
6 . 소노캄 제주
7 . 켄싱턴리조트 서귀포
8 . 롯데호텔 제주
9 . 금호제주리조트
10 . 메종글래드제주
11 . 롯데시티호텔 제주
12 . 제주신화월드 서머셋
13 . 호텔리젠트마린
14 . 해비치 호텔&리조트 제주
15 . 호텔시리우스
1 . 제주부영호텔앤리조트
2 . 제주신화월드 랜딩관
3 . 골든튤립제주성산호텔
4 . 탐라스테이호텔제주
5 . 히든클리프호텔&네이쳐
6 . 제주신라호텔
7 . WE호텔
8 . 코델리아S호텔
9 . 코업시티호텔 하버뷰
10 . 토스카나호텔
11 . 다인오세아노호텔
12 . 제주신화월드 메리어트관
13 . 휘슬락호텔
14 . 블루하와이리조트
15 . 에코그린리조트
1 . 화조원
2 . 캐니언파크 제주점
3 . 새별프렌즈
4 . 라온쥬
5 . 개똥이동물원
not found
1 . 더마파크
2 . 조랑말체험공원
3 . 탐라승마장
4 . 중문승마공원
5 . 제주승마공원
1 . 제주조랑말타운
2 . 조은승마장
3 . OK승마장
4 . 홀스스토리승마장
5 . 운주승마클럽
6 . 산도스카발리오홀스파크
7 . 세리승마
8 . 웅지승마센터
9 . 공룡투어 승마장
10 . 아덴힐승마클럽
11 . 성읍승마장
12 . 힐링팜 농어촌승마교육장
13 . 홀스홀릭포니클럽
14 . 제이앤승마클럽
